In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.chdir(globals()['_dh'][0])
os.chdir('..')
print(os.path.abspath(os.curdir))

In [2]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import random 
import tensorflow as tf
from tqdm import tqdm
from scipy import signal as sg

import rfcutils # this corresponds to utility functions provided for the challenge

import rfcutils.ofdm_helper_fn_short as ofdmfn

from src import unet_model as unet
from src.time_proc import long_window

get_sinr = lambda s, i: 10*np.log10(np.mean(np.abs(s)**2)/np.mean(np.abs(i)**2))
get_pow = lambda s: np.mean(np.abs(s)**2)

2022-08-17 08:21:16.553524: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


---

In [3]:
def eval_qpsk_demod(sig_est, sig_true, start_idx):
    qpsk_sig_idx = 16 - start_idx%16
    bit_est = rfcutils.matched_filter_demod(sig_est[qpsk_sig_idx:])
    bit_true = rfcutils.matched_filter_demod(sig_true[qpsk_sig_idx:])
    ber = np.sum(bit_est != bit_true)/len(bit_true)
    assert len(bit_est) == len(bit_true)
    return ber

def eval_sigs(sig_est, sig_true, start_idx):
    mse = get_pow(sig_est - sig_true)
    mse_db = 10*np.log10(mse)
    sdr = get_sinr(sig_true, sig_est-sig_true)
    ber = eval_qpsk_demod(sig_est, sig_true, start_idx)
    return (mse, mse_db, sdr, ber)

## Load Data

In [30]:
all_sig_mixture, all_sig1, all_sig2, all_sync_idx2, all_sig1_syms, all_sig1_bits, all_start_idx2 = pickle.load(open('dataset/Ex3C3_QPSK_OFDMQAM16.pickle','rb'))
all_val_sig_mixture, all_val_sig1, all_val_sig2, all_val_sync_idx2, all_val_sig1_syms, all_val_sig1_bits, all_val_start_idx2 = pickle.load(open('dataset/Ex3C3_ValSet_QPSK_OFDMQAM16.pickle','rb'))


## Parameters

In [31]:
sig_len_qpsk = 40960 # This is equal to 40960*6/8. Needed for the decimation
all_sinr = np.arange(-30, 4, 1.5)
random.seed(3)
np.random.seed(3)
tf.random.set_seed(3)
n_per_sinr_tr = 1000
n_per_sinr_val = 100
training_examples = 900
val_examples = 100
target_snr = 10
window_len = 40960
seg_len = 0


## Process data

In [39]:
all_sig_mixture_w, all_sig1_w, all_sig2_w = [], [] , []
for j in tqdm(np.arange(len(all_sinr))):
    for i in range(training_examples):
        target_sinr = all_sinr[j]
        start_idx2 = np.array(all_sync_idx2[j*n_per_sinr_tr + i])
        all_sig1_ex = np.array(all_sig1[j*n_per_sinr_tr + i])
        sig1 = all_sig1_ex[:window_len]
        all_sig2_ex = np.array(all_sig2[j*n_per_sinr_tr + i])
        sig2 = all_sig2_ex[:window_len]
        tau_b = (80 - start_idx2%80)%80
        
        CNnoise = np.empty(sig1.shape, dtype=np.complex128)
        CNnoise.real = np.random.normal(size=sig1.shape)/np.sqrt(2)
        CNnoise.imag = np.random.normal(size=sig1.shape)/np.sqrt(2)
        
        coeff = np.sqrt(np.mean(np.abs(sig1)**2)/(np.mean(np.abs(sig2)**2)*(10**(target_sinr/10))))
        coeff_noise = np.sqrt(np.mean(np.abs(sig1)**2)/(np.mean(np.abs(CNnoise)**2)*(10**(target_snr/10))))

        sig_mixture = sig1 + sig2 * coeff + CNnoise * coeff_noise        
        
        all_sig_mixture_w.append(sig_mixture)
        all_sig1_w.append(sig1)

for j in tqdm(np.arange(len(all_sinr))):
    for i in range(val_examples):
        target_sinr = all_sinr[j]
        start_idx2 = np.array(all_val_sync_idx2[j*n_per_sinr_val + i])
        all_sig1_ex = np.array(all_val_sig1[j*n_per_sinr_val + i])
        sig1 = all_sig1_ex[:window_len]
        all_sig2_ex = np.array(all_val_sig2[j*n_per_sinr_val + i])
        sig2 = all_sig2_ex[:window_len]
        tau_b = (80 - start_idx2%80)%80
        
        CNnoise = np.empty(sig1.shape, dtype=np.complex128)
        CNnoise.real = np.random.normal(size=sig1.shape)/np.sqrt(2)
        CNnoise.imag = np.random.normal(size=sig1.shape)/np.sqrt(2)
        
        coeff = np.sqrt(np.mean(np.abs(sig1)**2)/(np.mean(np.abs(sig2)**2)*(10**(target_sinr/10))))
        coeff_noise = np.sqrt(np.mean(np.abs(sig1)**2)/(np.mean(np.abs(CNnoise)**2)*(10**(target_snr/10))))

        sig_mixture = sig1 + sig2 * coeff + CNnoise * coeff_noise
        
        all_sig_mixture_w.append(sig_mixture)
        all_sig1_w.append(sig1)
        
all_sig_mixture_w = np.array(all_sig_mixture_w)
all_sig1_w = np.array(all_sig1_w)

100%|██████████| 23/23 [00:08<00:00,  2.73it/s]


## Train

In [ ]:
sig1_out = all_sig1_w.reshape(-1,window_len)
out1_comp = np.dstack((sig1_out.real, sig1_out.imag))

all_mixture_seg = long_window(all_sig_mixture_w, window_len, seg_len)
mixture_bands_comp = np.dstack((all_mixture_seg.real, all_mixture_seg.imag))

mixture_input_nn = np.dstack((mixture_bands_comp, mixture_bands_comp))

print(f'Output shape: {out1_comp.shape}; Input shape: {mixture_input_nn.shape}')

long_k_sz = 101
model_name = f'ofdm_{window_len}_K{long_k_sz}_XL_TS1000_sync_4in_noisy_{target_snr}_m0'
print(f'Training {model_name}')
nn_model = unet.get_unet_model_XL_4((window_len+seg_len, 4), k_sz=3, long_k_sz=long_k_sz, start_idx=seg_len//2, window_len=window_len)

checkpoint_filepath = f'./tmp_checkpoints/ofdm_{window_len}_K{long_k_sz}_XL_TS1000_sync_4in_noisy_{target_snr}_m0'
IR_model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=200)

nn_model.fit(mixture_input_nn, out1_comp, epochs=2000, batch_size=32, validation_split=0.1,shuffle=True, verbose=1,callbacks=[stop_early,IR_model_checkpoint_callback])


Training ofdm_40960_K101_XL_TS1000_sync_4in_noisy_10_m0
Epoch 1/2000
646/647 [============================>.] - ETA: 0s - loss: 0.3396